In [ ]:
!pip install opencv-python
!pip install moviepy

In [94]:
import cv2
import numpy as np
from google.colab import drive
from moviepy.editor import VideoFileClip
import matplotlib.pyplot as plt

drive.mount('/content/drive')

def detect_cars(image):
    # Make a copy of the image to avoid read-only issues
    image_copy = image.copy()

    # Convert to RGB for displaying
    image_rgb = cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image_copy, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)

    # Detect edges using Canny edge detection
    edges = cv2.Canny(blurred, 30, 150)

    # Find contours in the edge image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate through the contours and filter out non-car-like shapes
    cars = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 50 and h > 50:
            cars.append((x, y, w, h))

    # Draw bounding boxes around the detected cars
    for car in cars:
        x, y, w, h = car
        cv2.rectangle(image_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Draw the edges on the original image
    edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    image_with_edges = cv2.addWeighted(image_copy, 0.8, edges_colored, 1, 0)

    return image_with_edges

def detect_lines(image):
    # Make a copy to avoid read-only issues
    image_copy = image.copy()

    # Define the coordinates for cropping
    x_start, y_start = 0, 0
    x_end, y_end = 3080, 1750

    # Crop the image
    cropped_image = image_copy[y_start:y_end, x_start:x_end]

    # Convert to grayscale
    gray = cv2.cvtColor(cropped_image, cv2.COLOR_RGB2GRAY)

    # Apply Gaussian blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Detect edges using Canny
    edges = cv2.Canny(blur, 50, 150)

    # Define region of interest (ROI)
    height, width = edges.shape[:2]
    roi_vertices = np.array([[(0, height), (width / 2, height / 2), (width, height)]], dtype=np.int32)
    mask = np.zeros_like(edges)
    cv2.fillPoly(mask, roi_vertices, 255)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Apply Hough transform to detect lines
    lines = cv2.HoughLinesP(masked_edges, rho=1, theta=np.pi/180, threshold=50, minLineLength=100, maxLineGap=50)

    # Draw detected lines on the cropped image
    line_image = np.zeros_like(cropped_image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 5)

    # Combine the original image with the detected lines
    result = cv2.addWeighted(cropped_image, 0.8, line_image, 1, 0)

    return result

def combined_processing(image):
    # First, detect cars
    image_with_cars = detect_cars(image)
    # Then, draw edges and lines
    final_image = detect_lines(image_with_cars)
    return final_image

# Example usage with VideoFileClip
video_path = '/content/drive/My Drive/Car Video/video_input.mp4'
clip = VideoFileClip(video_path)
processed_clip = clip.fl_image(combined_processing)

# Optionally save the processed video
output_path = '/content/drive/My Drive/Car Video/processed_video.mp4'
processed_clip.write_videofile(output_path, codec='libx264')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Moviepy - Building video /content/drive/My Drive/Car Video/processed_video.mp4.
MoviePy - Writing audio in processed_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/My Drive/Car Video/processed_video.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/My Drive/Car Video/processed_video.mp4
